In [12]:
# Import 設定
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parents[0]
sys.path.insert(0, PROJECT_DIR.joinpath("src").__str__())  # 不能用 pathlib.PosixPath 傳入

# Step 01: Ingestion

In [2]:
from ingestion.file_loaders.goodnotes.loader import GoodnotesLoader
loader = GoodnotesLoader()

loader_results = []
for filepath in ["早安日語-孫寅華.pdf",]:
    print(filepath)
    loader_results = loader.load(filepath)

/home/kevinwang/.local/share/virtualenvs/personal-rag-7yCtySkO/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:717: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/kevinwang/.paddlex/official_models/PP-OCRv5_server_det`.
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/kevinwang/.paddlex/official_models/PP-OCRv5_server_rec`.
[2025-08-30 18:52:48,173] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 388 0 (offset 0)
[2025-08-30 18:52:48,173] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 390 0 (offset 0)
[2025-08-30 18:52:48,173] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 392 0 (offs

早安日語-孫寅華.pdf


[2025-08-30 18:52:48,290] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1308 0 (offset 0)
[2025-08-30 18:52:48,290] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1310 0 (offset 0)
[2025-08-30 18:52:48,291] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1312 0 (offset 0)
[2025-08-30 18:52:48,291] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1314 0 (offset 0)
[2025-08-30 18:52:48,291] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1316 0 (offset 0)
[2025-08-30 18:52:48,291] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1318 0 (offset 0)
[2025-08-30 18:52:48,292] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1320 0 (offset 0)
[2025-08-30 18:52:48,292] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1322 0 (offset 0)
[2025-08-30 18:52:48,292] [ WARNING] _utils.py:457 - Ignoring wrong pointing object 1324 0 (offset 0)
[2025-08-30 18:52:48,292] [ WARNING] _utils.py:457 - Ignoring wrong pointing objec

In [3]:
loader_results[:100]

[LoaderResult(content='早安日語\n孫寅華', metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf', title=None, author=None, subject=None, created_at=datetime.datetime(2025, 8, 30, 10, 50, 4, tzinfo=TzInfo(UTC)), modified_at=datetime.datetime(2025, 8, 30, 10, 50, 4, tzinfo=TzInfo(UTC)), is_chunk=False, chunk_info=None, extra={'bg_mode': 'black'}, source=None, producer='iOS Version 18.5 (Build 22F76) Quartz PDFContext', page=1, outlines=[]), doc=None),
 LoaderResult(content='早安日語\n孫寅華\n\n二\n▲\n★\n●\n0\n?\n\n\n-\n1', metadata=GoodnotesMetadata(file_type=<FileType.PDF: 'pdf'>, file_name='早安日語-孫寅華.pdf', title=None, author=None, subject=None, created_at=datetime.datetime(2025, 8, 30, 10, 50, 4, tzinfo=TzInfo(UTC)), modified_at=datetime.datetime(2025, 8, 30, 10, 50, 4, tzinfo=TzInfo(UTC)), is_chunk=False, chunk_info=None, extra={'bg_mode': 'black'}, source=None, producer='iOS Version 18.5 (Build 22F76) Quartz PDFContext', page=1, outlines=[]), doc=None),
 LoaderResult(co

# Step 2: Chunk

In [22]:
from chunking.no_chunk import NoChunkProcessor
chunker = NoChunkProcessor()

chunks = []
for loader_result in loader_results:
    if not loader_result.content:
        continue
    chunks += chunker.process(doc=loader_result.content,
                              metadata=loader_result.metadata,
                              )

### 手動將 Outline 加入 Chunk

In [ ]:
# for idx in range(len(chunks)):
#     chunks[idx].content = "\n".join(chunks[idx].metadata.outlines) + "\n\n" + chunks[idx].content

### 手動調整頁碼

In [ ]:
# import re
# for idx in range(len(chunks)):
#     filename = chunks[idx].metadata.file_name
#     if filename.startswith("早安日語-孫寅華"):
#         regex = re.search(r"早安日語-孫寅華part(\d+)-page(\d+)", filename)
#         if regex:
#             part_num = int(regex.group(1))
#             page_num = int(regex.group(2))
#     chunks[idx].metadata.page += (part_num-1)*page_num


In [23]:
len(chunks)

472

In [24]:
print(chunks[0]._raw_chunk)

None


/tmp/ipykernel_12209/3635813002.py:1: UserWarning: _raw_chunk 為備份欄位，僅應在 chunking 階段或 debugging 時使用
  print(chunks[0]._raw_chunk)


# Step 3: Embedding

In [30]:
import os
from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel


embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
for idx in range(len(chunks)):
    vector = embedder.encode(chunks[idx].content)
    chunks[idx].embedding = vector

# Step 04: Insert

## Vector-based

In [31]:
from infra.stores.pgvector import PGVectorStore

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )

for chunk in chunks:
    vec_store.insert(chunk)

## Lexical-based

In [32]:
from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )

for chunk in chunks:
    lex_store.insert(chunk)

# Step 05: Retrieve

## Vector-based

In [33]:
import os

from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel
from infra.stores.pgvector import PGVectorStore

embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
query_vector = embedder.encode("自動詞與他動詞的判斷邏輯")

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )
results = vec_store.search(query_vector)

In [34]:
[_.chunk.content for _ in results]

['(lesson100~101)\nO\nO\nO\n自動詞\n為何用在，不用\n2.色比仁多了命令，强利意味\n2.避免仁重覆使用\nO',
 '形動詞是並列關係，可調換\nEx:日本は便利了綺震区国です。\n結構同5.\n 形容詞，形動詞是因果關係，不可調換\n形容詞+形動詞\n由於「難)」和大变有因果明係，所以不能調換\nRemark\n：此處的で與詞性無關，為兩個句子合併。\nFor exomple, (66)\n日本はされいで便利な国です。\n日本は便利な国です。\n(lesson 21~22)\n連体形修飾後方名詞\n若(形動)無法直接形容到主詞\n則會用力做為助詞連接\n常发生的有上手。“下手，好老，“嫌\n(2)的否定形\n(父親開車的技術不差)\n中止形表同褒贬，並列。\n(形動)+(形動)\n形容詞\n形容動詞/名詞：\nRemark：中止形\n(形)+(形動)\n(形動>(形)',
 '(Channel+ 早安日語，豫寅華，lesson 20~23)\n(lesson20)\n*加强\n或稱形動詞\n(1)形容詞\n在形容詞\nT $\n(\n\\\n…)\nです\nです\n<名，形動>笨拙，不高明\n《名;形動>討厭，嫌惡\n〈形動)喜歡，愛好\n(形動)安静\n<形動>高明的，拿手的\n静か\n*\n原形：\n修節名詞：\n敬体：\n<名;形動>親切\n(形動>漂亮\n<形動>嚴重，不得了.辛苦\n(也可做“副词)\n(形動>氣派，華麗，宏偉，有為\n(形)年輕的\n<名)駕駅(行高)\n(名)地方、處所\n(形動>有名\n<形動>熟闹\nexample\n(形動>方便\n静がです\n二微\n*交，為“謙稱，在”對外人提及自己親时使用\nお父之人，敬，用和自人說用\n街，的用途很，可指“市，即镇，街道，\n*転駕験的行：運手：司机\n前華，學長\n<名>學習\n(名)城镇\n<名)料理.菜餚\n(名)學科\n<名)助手，(大學)助教\n副>尚，還\n(連体>(不定詞)什麼樣的？如何的？\n(lesson 2~21)\n：强調後方内容\n「が」：強調前方主題\nNote:廖老師對は，雨個助詞的教學\n*主詞+は+形動詞形動詞若能直接，修飾主詞則用は」\n若無法直接，形容到主詞，則是用が，連接\nEx:上手.下手，好艺，嫌.\n我很擅長日文私

## Lexical-based

In [37]:
import os

from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )
results = lex_store.search("つつある")

In [38]:
[_.chunk.content for _ in results]

['よみもの\nはい\nにほんじん\nみ\nこたつに入ってみかんを食べながら、テレビを見るこれは日本人の\n不ゆ\nす\nかた\nだいひようてき\nき\n（\nふとん\nうえ\nた代表的な冬の過ごし方。こたつ\n木を組んぐの上に布団ぶ\n*注意此處ガぶせるv.s.のせる\nつくえいた\nほ\nなかだんほうきく\nい\nガぶせる：從上往下罩住\n→のせる：放在上面\nせて、机の板をのせて、中に暖房器具を入れたものです。こたつには掘り\nお\nにしゆるい\nゆかひくほ\nほ\nごたつと置きごたつの二種類があります。掘りごたつは、床を低く掘って\nあし\nなかたんぼうき\nの\nはい\n中に暖房器具を入れます。足を伸ばしたま、こたつに入ることができま\n記下\nつか\nなつあいだ\nいま\nいた\nす。夏の間こたつを使わないときには、板をかぶせます。しかし、今では\nつかひとほお\nてかる。\nお\nお\nより手軽な置きごたつを使う人の方が多いです。置きごたつには、やぐら\n(副)：更~(程度更甚)\nだんぼうきぐ\nうら\nの裏に暖房器具がついています。\nへやぜんたいあたた\nからだ、あたた\nこたつは体を暖めることができますが、部屋全体を暖めることができま\n使~暖和\nげんさい\nひじようさむちほう\nつか\nほっかいどう\nせん。ですから、北海道など非常に寒い地方ではあまり使いません。現在\nだんぼうきしゆいおお\nでんき\nはヒーター、ストープ、電気カーペットなど暖房器具の種類が多くなりま\nあたた\nばしよ\nあい\nかぞくあつ\nした。しかし、こたつは家族が集まってくつろいで暖まる場所として、相\n<自五>表\nか\nにんき\n変わらず人気があります。\n有人気\n<副)仍然\n置きごたつ\n掘りごたつ\n机の板\n·布団·\n暖房器具\nCD1-21\n37',
 'インターチェンジ\nはい\nあめ\nふ\nつづ\nむあつ\nかん\nつゆに入って雨が降り続くと、蒸し暑くてしい感じになる。\nとき\nぼうず\nつく\nつき\nねが\nひ\nせいてん\nそんな時は、「てる坊主」を作っ、次の日の晴天をお願いしてみ\nぼうずかみぬのつく\nあし\nにんぎょう\nよう。てるてる坊主は紙か布で作れた、手も足もない人形こと。\nこども\nせいてん\nねが\nあしたてん